<center>

<img src="https://i.imgur.com/Av86m7R.png" alt="logo" width="100%" height="270">
<h1><b>Machine Learning<b></h1>
<h2>Session 6 - Metrics & Losses</h1>

**&copy; 2023 CoTAI. Internal Use Only.**

</center>


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/train_data.zip" -d "/content"

Streaming output truncated to the last 5000 lines.
  inflating: /content/train_data/S/190.jpg  
  inflating: /content/train_data/S/1900.jpg  
  inflating: /content/train_data/S/1901.jpg  
  inflating: /content/train_data/S/1902.jpg  
  inflating: /content/train_data/S/1903.jpg  
  inflating: /content/train_data/S/1904.jpg  
  inflating: /content/train_data/S/1905.jpg  
  inflating: /content/train_data/S/1906.jpg  
  inflating: /content/train_data/S/1907.jpg  
  inflating: /content/train_data/S/1908.jpg  
  inflating: /content/train_data/S/1909.jpg  
  inflating: /content/train_data/S/191.jpg  
  inflating: /content/train_data/S/1910.jpg  
  inflating: /content/train_data/S/1911.jpg  
  inflating: /content/train_data/S/1912.jpg  
  inflating: /content/train_data/S/1913.jpg  
  inflating: /content/train_data/S/1914.jpg  
  inflating: /content/train_data/S/1915.jpg  
  inflating: /content/train_data/S/1916.jpg  
  inflating: /content/train_data/S/1917.jpg  
  inflating: /content/train_dat

In [ ]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 12.3 MB/s eta 0:00:00


In [ ]:
import mediapipe as mp
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import os
from pathlib import Path
import tensorflow as tf
batch_size_model = 32

# Image resolution for NN
img_height = 64
img_width = 64

# Data paths
data_dir = "/content/train_data"
class_names = os.listdir(data_dir)

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size_model)

Found 18000 files belonging to 6 classes.


In [ ]:
from tensorflow.keras.layers import  RandomBrightness, RandomFlip,  RandomRotation,  RandomZoom
from tensorflow.keras import layers, Sequential
data_augmentation = Sequential([
  RandomFlip("horizontal"),
  RandomRotation(0.1),
  RandomZoom(0.1),
  RandomBrightness(0.2)])

train_ds = train_ds.map(lambda x, y: (data_augmentation(x), y))

In [ ]:
class CustomHands(mp.solutions.hands.Hands):
    def __init__(self, static_image_mode=False, min_detection_confidence=0.5, **kwargs):
        max_num_hands = 1
        super().__init__(
            self, static_image_mode, max_num_hands, min_detection_confidence, **kwargs
        )
    def process(self, path: str):
        img = self.__read_img(path)
        results = super().process(img)
        if not results.multi_hand_world_landmarks:
            return None
        for hand_world_landmarks in results.multi_hand_world_landmarks:
            hand = (
                np.array(
                    [[res.x, res.y, res.z] for res in hand_world_landmarks.landmark]
                )
                .transpose()
                .flatten()
            )
        return hand
    def process_all(self, rootdir: str):
        labels = os.listdir(rootdir)
        labels.sort()
        X = list()
        y = list()

        for label in labels:
            list_img_paths = [os.path.join(rootdir, label, img) for img in os.listdir(os.path.join(rootdir, label))]
            for path in list_img_paths:
                landmarks = self.process(path)
                if landmarks is None:
                    continue
                X.append(landmarks[np.r_[0:3, 15:18, 24:27, 27:30, 36:39, 39:42, 48:51, 51:54, 57:60, 60:63]]) #Các Landmarks chính để nhận diện 0, 5, 8, 9, 12, 13, 16, 17, 20 và 21
                y.append(label)

            print(f"{label} finished")
        return np.vstack(X), np.asarray(y)

    def __read_img(self, d: str):
        img = cv2.imread(d)
        img = cv2.flip(img, 1)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img

In [ ]:
  hands=CustomHands(static_image_mode=True,
                  min_detection_confidence=0.5)
  X,y = hands.process_all(data_dir)

A finished
D finished
E finished
Q finished
S finished
W finished


In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
num_classes = len(class_names)
le = LabelEncoder()
y_encode = le.fit_transform(y)
y_encode = to_categorical(y_encode, num_classes)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encode, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test=train_test_split(X_test, y_test, test_size=0.5, random_state=42)

X_train = np.reshape(X_train, (len(X_train), 1, 30))
X_val = np.reshape(X_val, (len(X_val), 1, 30))
X_test = np.reshape(X_test, (len(X_test), 1, 30))
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size_model)
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size_model)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size_model)
input_shape=((1, X_train.shape[1],))

In [ ]:
from tensorflow import keras
callback = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
model = Sequential()
model.add(layers.LSTM(512, return_sequences=True, input_shape=(1,30)))
model.add(layers.LSTM(512, return_sequences=True))
model.add(layers.LSTM(256, return_sequences=True))

model.add(layers.LSTM(256))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))
model.compile(loss='CategoricalCrossentropy', optimizer='adam', metrics=["accuracy"])

In [ ]:
model.fit(train_ds, validation_data=val_ds, epochs=100, callbacks = [callback])
model.save('model.h5')

Epoch 1/100
443/443 [==============================] - 20s 13ms/step - loss: 0.8201 - accuracy: 0.6183 - val_loss: 0.0556 - val_accuracy: 0.9836
Epoch 2/100
443/443 [==============================] - 6s 13ms/step - loss: 0.0245 - accuracy: 0.9942 - val_loss: 0.0431 - val_accuracy: 0.9859
Epoch 3/100
443/443 [==============================] - 4s 10ms/step - loss: 0.0077 - accuracy: 0.9987 - val_loss: 0.0206 - val_accuracy: 0.9938
Epoch 4/100
443/443 [==============================] - 4s 10ms/step - loss: 0.0445 - accuracy: 0.9933 - val_loss: 0.0056 - val_accuracy: 0.9989
Epoch 5/100
443/443 [==============================] - 6s 14ms/step - loss: 0.0057 - accuracy: 0.9992 - val_loss: 0.0050 - val_accuracy: 0.9983
Epoch 6/100
443/443 [==============================] - 5s 10ms/step - loss: 0.0046 - accuracy: 0.9992 - val_loss: 0.0027 - val_accuracy: 0.9989
Epoch 7/100
443/443 [==============================] - 5s 11ms/step - loss: 0.0390 - accuracy: 0.9902 - val_loss: 0.0119 - val_accuracy